# Main Dishes, American, Pull #1

In [1]:
import json
import requests
import pandas as pd
import numpy as np 
import re

In [2]:
# Course search terms
allowedCourse = requests.get('http://api.yummly.com/v1/api/metadata/course?_app_id=e2b9bebc&_app_key=4193215272970d956cfd5384a08580a9')

# Cuisine search terms
allowedCuisine = requests.get('http://api.yummly.com/v1/api/metadata/cuisine?_app_id=e2b9bebc&_app_key=4193215272970d956cfd5384a08580a9')

In [3]:
# Slice crap off beggining and end
response_course = allowedCourse.text[23:-2]
response_cuisine = allowedCuisine.text[24:-2]

# Print search terms
course_list = json.loads(response_course)
cuisine_list = json.loads(response_cuisine)
print [x['searchValue'] for x in course_list]
print [x['searchValue'] for x in cuisine_list]

[u'course^course-Main Dishes', u'course^course-Desserts', u'course^course-Side Dishes', u'course^course-Appetizers', u'course^course-Salads', u'course^course-Breakfast and Brunch', u'course^course-Breads', u'course^course-Soups', u'course^course-Beverages', u'course^course-Condiments and Sauces', u'course^course-Cocktails', u'course^course-Snacks', u'course^course-Lunch']
[u'cuisine^cuisine-american', u'cuisine^cuisine-kid-friendly', u'cuisine^cuisine-italian', u'cuisine^cuisine-asian', u'cuisine^cuisine-mexican', u'cuisine^cuisine-southern', u'cuisine^cuisine-french', u'cuisine^cuisine-southwestern', u'cuisine^cuisine-barbecue-bbq', u'cuisine^cuisine-indian', u'cuisine^cuisine-chinese', u'cuisine^cuisine-cajun', u'cuisine^cuisine-mediterranean', u'cuisine^cuisine-greek', u'cuisine^cuisine-english', u'cuisine^cuisine-spanish', u'cuisine^cuisine-thai', u'cuisine^cuisine-german', u'cuisine^cuisine-moroccan', u'cuisine^cuisine-irish', u'cuisine^cuisine-japanese', u'cuisine^cuisine-cuban',

In [278]:
url = 'http://api.yummly.com/v1/api/recipes?'
headers = {'X-Yummly-App-ID':'e2b9bebc', 'X-Yummly-App-Key':'4193215272970d956cfd5384a08580a9'}

In [279]:
parameters = {'allowedCourse[]': 'course^course-Main Dishes', 
              'excludedCourse[]': ['course^course-Appetizers', 'course^course-Salads', 
                                   'course^course-Condiments and Sauces',
                                   'course^course-Lunch', 'course^course-Soups', 'course^course-Snacks',
                                   'course^course-Breakfast and Brunch', 'course^course-Side Dishes',],
              'allowedCuisine[]': 'cuisine^cuisine-asian',
              'excludedCuisine[]': ['cuisine^cuisine-american', 'cuisine^cuisine-italian', 'cuisine^cuisine-indian',
                                    'cuisine^cuisine-mexican', 'cuisine^cuisine-mediterranean', 'cuisine^cuisine-chinese',
                                   'cuisine^cuisine-japanese'],
              'maxResult': 500,
             'start': 1500}

In [280]:
response = requests.get(url, headers=headers, params=parameters)
response.status_code

200

In [281]:
# Decode JSON
api_call = response.json()
print type(api_call)

# Print keys
response_keys = api_call.keys()
print response_keys

# Print total match count
print api_call['totalMatchCount']

<type 'dict'>
[u'matches', u'totalMatchCount', u'attribution', u'facetCounts', u'criteria']
11722


In [282]:
#forming lists to create dataframes of the features we want. 
main_list = []
ingredients_list = []
attributes_list = []
flavors_dict = {}

for food in api_call['matches']:

    _d1 = {}
    _d1['id'] = food['id']
    _d1['rating'] = food['rating']
    _d1['recipeName'] = food['recipeName']
    _d1['sourceDisplayName'] = food['sourceDisplayName']
    
    main_list.append(_d1)
    
    _d2 = {}
    _d2['id'] = food['id']
    _d2['course'] = 'Main Dish'
    _d2['ingredient_list'] = food['ingredients']
    for i in food['ingredients']:
        i = i.lower() # additional code to conver to lowercase
        i = re.sub(r'\d+%\s', '', i) # additional code to remove 1%, 2%, etc
        i = re.sub(r'\xae', '', i) # remove '\xae' characters
        i = re.sub(r'shredded\s', '', i)
        i = re.sub(r'chopped\s', '', i)
        i = re.sub(r'diced\s', '', i)
        i = re.sub(r'crumbled\s', '', i)
        i = re.sub(r'fresh\s', '', i)
        i = re.sub(r'grated\s', '', i)
        i = re.sub(r'fat free\s', '', i)
        i = re.sub(r'boneless\s', '', i)
        i = re.sub(r'boneless skinless\s', '', i)
        i = re.sub(r'minced\s', '', i)
        i = re.sub(r'sliced\s', '', i)
        i = re.sub(r'(?!ground beef)ground ', '', i)
        i = re.sub(r'^dried\s', '', i)
        i = re.sub(r'^cooked\s', '', i)
        
        _d2[i] = 1
    ingredients_list.append(_d2)

    _d3 = {}
    _d3['id'] = food['id']
    for k, v in food['attributes'].items():
        for i in v:
            _d3[i] = 1
    attributes_list.append(_d3)
    

    flavors_dict[food.get('id')] = food.get('flavors')

In [283]:
# read in dictionary for course and cuisine and create list of possible values for each
course_df = pd.read_csv('../datasets/master_courses.csv', names=['course'])
cuisine_df = pd.read_csv('../datasets/master_cuisines.csv', names=['cuisine'])

course_list = course_df.course
cuisine_list= cuisine_df.cuisine

In [284]:
#create dictionary of cuisine and course for each recipe
course_dict={}
cuisine_dict={}
for food in api_call['matches']:
    course_dict[food.get('id')]= food['attributes'].get('course')
    cuisine_dict[food.get('id')]= food['attributes'].get('cuisine')
    
_courses= {}

for k, v in course_dict.iteritems():
    course_val = {}
    for course in course_list:
        try:
            if course in v :
                course_val[course] = 1
            else:
                course_val[course] = 0
        except TypeError:
            course_val[course] = 0
    
        _courses[k] = course_val
        
_cuisines= {}       

for k, v in cuisine_dict.iteritems():
    cuisine_val = {}
    for course in cuisine_list:
        try:
            if course in v :
                cuisine_val[course] = 1
            else:
                cuisine_val[course] = 0
        except TypeError:
            cuisine_val[course] = 0
    
        _cuisines[k] = cuisine_val

## Recipe Detail Query

In [285]:
#get list of recipe ids
recipe_ids=[]
for recipe in api_call['matches']:
    recipe_ids.append(recipe['id'])

In [286]:
# second api call to get other features for each recipe
key_id= '_app_id=e2b9bebc&_app_key=4193215272970d956cfd5384a08580a9'
url='http://api.yummly.com/v1/api/recipe/'

In [287]:
# retrieve other features for all recipes

def get_recipe(_id):
    response = requests.get(url + _id + '?' + key_id)
    return response.json()

recipes=[]
for _id in recipe_ids :
    recipes.append(get_recipe(_id))

In [288]:
#for each recipe create a new dictionary of selected attributes and append into a list

recipe_details=[]
for recipe in recipes:
    _dict={}
    _dict['id']=recipe['id']
    _dict['ingredientCount']= len(recipe['ingredientLines'])
    _dict['numberOfServings']= recipe['numberOfServings']
    _dict['prepTimeInSeconds'] = recipe.get('prepTimeInSeconds')
    _dict['cookTimeInSeconds'] = recipe.get('cookTimeInSeconds')
    _dict['totalTimeInSeconds']= recipe.get('totalTimeInSeconds')
    
    
    recipe_details.append(_dict)

In [289]:
#create dataframes, arrange column index and save into csv
# df_main = pd.DataFrame(main_list)
# df_main.to_csv('asian/main_as4.csv', encoding ='utf-8')

# df_ingredients = pd.DataFrame(ingredients_list)
# df_ingredients = df_ingredients.fillna(0)
# cols = list(df_ingredients)
# cols.insert(0, cols.pop(cols.index('id')))
# cols.insert(1, cols.pop(cols.index('course')))
# df_ingredients= df_ingredients.ix[:,cols]
# df_ingredients.to_csv('asian/main_as_ingredients4.csv', encoding ='utf-8')

# df_attributes = pd.DataFrame(attributes_list)
# df_attributes = df_attributes.fillna(0)
# cols = list(df_attributes)
# cols.insert(0, cols.pop(cols.index('id')))
# df_attributes = df_attributes.ix[:,cols]
# df_attributes.to_csv('asian/main_as_attributes4.csv', encoding= 'utf-8')

# df_flavors = pd.DataFrame(flavors_dict).transpose()
# df_flavors.reset_index(level=0, inplace=True)
# df_flavors.to_csv('asian/main_as_flavors4.csv', encoding= 'utf-8')

# df_courses = pd.DataFrame(_courses).transpose()
# df_courses.reset_index(level=0, inplace=True)
# df_courses.to_csv('asian/main_as_courses4.csv', encoding= 'utf-8')

# df_cuisines = pd.DataFrame(_cuisines).transpose()
# df_cuisines.reset_index(level=0, inplace=True)
# df_cuisines.to_csv('asian/main_as_cuisines4.csv', encoding= 'utf-8')

# df_details=pd.DataFrame(recipe_details)
# cols = list(df_details)
# cols.insert(0, cols.pop(cols.index('id')))
# df_details=df_details.ix[:,cols]
# df_details.to_csv('asian/main_as_details4.csv', encoding= 'utf-8')